In [ ]:
include("HighDimMixedModels.jl")

In [ ]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
#import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [ ]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict(:a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)

In [ ]:
df

In [ ]:
## https://stackoverflow.com/questions/28756206/julia-converting-column-type-from-integer-to-float64-in-a-dataframe
#df[:,:a] = convert(Vector{Float64},df[:,2]) # assign vector values and convert them into Int64
df[!,:a] = convert(Vector{Float64},df[:,2]) # change the vector reference to the vector with float64
first(df,5)

In [ ]:
for i in 1:size(df,2)
    if(eltype(df[:,i]) == Int64)
        println("y")
        df[!,i] = convert(Vector{Float64},df[:,i])
    end
end

In [ ]:
df

In [ ]:
f

**try for f2**

In [ ]:
# try for f2
f2 = @formula(y ~ 1 + a + b + (1|c))

In [ ]:
sch = schema(f2,df,contrasts)

In [ ]:
typeof(f2)

In [ ]:
form = apply_schema(f2, sch)

In [ ]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)

In [ ]:
form = MixedModels.apply_schema(f2, sch, HighDimMM.highDimMixedModel)

In [ ]:
?apply_schema

In [ ]:
typeof(LinearMixedModel)

In [ ]:
typeof(HighDimMM.highDimMixedModel)

**back here**

In [ ]:
sch = schema(f,df,contrasts)

In [ ]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)

In [ ]:
form = apply_schema(f, sch)

In [ ]:
form.rhs.terms[[2,3]]

In [ ]:
typeof(form)

In [ ]:
names(df)

In [ ]:
findall(x -> x == "a", names(df))

In [ ]:
if typeof(["a","b"]) == Array{String, 1}
    println("123")
else
    println("456")
end

In [ ]:
typeof(["a","b"])

In [ ]:
form.rhs.terms[[1,2]]

In [ ]:
form.rhs.terms

In [ ]:
modelcols(form.rhs.terms[3],df)

In [ ]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)
y,X = modelcols(form,df)

In [ ]:
X[2]

In [ ]:
modelcols(form.rhs.terms[1],df)

In [ ]:
Vector<:AbstractVector

In [ ]:
AbstractVecOrMat

In [ ]:
Vector(1:length(form.rhs.terms))

In [ ]:
[1,2]

In [ ]:
df[:,["b","c"]]

In [ ]:
y,pred = modelcols(form, df);

In [ ]:
y

In [ ]:
pred

In [ ]:
pred = modelmatrix(form, df);

In [ ]:
pred

In [ ]:
typeof(pred)

###### data cbpp

In [ ]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

In [ ]:
f = @formula(period ~ herd + incid + hsz)
contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

In [ ]:
sch = schema(f,df,contrasts)

In [ ]:
form = apply_schema(f, sch)

In [ ]:
form.rhs.terms

In [ ]:
modelcols(form.rhs.terms[3],df)

### Number of variables indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

In [ ]:
# Three matrix
HMM.M.M; HMM.X.X; HMM.Z.Z

### indices indicates M,X,Z

In [ ]:
include("HighDimMixedModels.jl")

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

In [ ]:
Matrix{Int64}<:AbstractVecOrMat{AbstractFloat}

In [ ]:
Matrix{Int64}<:AbstractVecOrMat{<:Number}

In [ ]:
[1.0 2.3; 2.0 4.9]::AbstractVecOrMat{Float64}

In [ ]:
Matrix<:AbstractVecOrMat

In [ ]:
HMM.M.M

In [ ]:
HMM.X.X

In [ ]:
HMM.M.M * HMM.X.X

In [ ]:
# response
y = HMM.y

## variable names indicates M,X,Z

In [ ]:
include("HighDimMixedModels.jl")

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")

In [ ]:
HMM.M.M

In [ ]:
HMM.X.X

In [ ]:
HMM.Z.Z

In [ ]:
y = HMM.y

In [ ]:
r = size(HMM.M,2) + size(HMM.X,2)

### optimization

In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10
  [508b3551] highDimMM v0.1.0 `~/.julia/dev/highDimMM`


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

In [3]:
using highDimMM
#using HighDimMixedModels

In [4]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  0 + a + b + (1|c) + d)
#contrasts = Dict(:a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)
df[!,:a] = convert(Vector{Float64},df[:,2]) # change the vector reference to the vector with float64
first(df,5)

,y,a,b,c,d
,Float64,Float64,Float64,String,Float64
1,0.585195,1.0,0.236782,d,0.752223
2,0.0773379,2.0,0.943741,e,0.314815
3,0.716628,3.0,0.445671,f,0.858522
4,0.320357,4.0,0.763679,d,0.698713
5,0.653093,5.0,0.145071,e,0.757746


In [20]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)
f = @formula(period ~ 0 + incid + hsz + (1|herd))

FormulaTerm
Response:
  period(unknown)
Predictors:
  0
  incid(unknown)
  hsz(unknown)
  (herd)->1 | herd

In [23]:
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ highDimMM /Users/zyxu/.julia/dev/highDimMM/src/bricks.jl:12


In [25]:
M = HMM.M.M
first(M,5)
M[1:5,:]

5×1 Matrix{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0

In [26]:
X = HMM.X.X
first(X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [49]:
Z = HMM.Z.Z
Z[1:5,:]
rank(Z)

15

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$

To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

In [29]:
y = HMM.y
first(y,5)

5-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 1.0

In [ ]:
# A is X in formula
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
C = transpose(u[:,1:4]) # simplify: assume fixed effect full rank
K = C*P

In [30]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = transpose(u[:,1:(n-r)]) 
C = randn((n-r),n)
K = C*P
Z = HMM.Z.Z
y = HMM.y
print()

In [31]:
maximum(K*A)

1.3233858453531866e-13

In [38]:
K

54×56 Matrix{Float64}:
  0.351461    1.03892     0.864582   …   1.50534     1.13175    -1.93464
 -0.378315   -1.34525     0.400791       0.503205    1.36433    -1.58355
  0.158502   -0.506437   -0.182473      -0.30453    -0.403104   -0.350821
  0.412256    0.55857     0.573899      -0.33975    -0.8374      0.143684
 -0.486077    0.136467   -1.22109        0.620559    1.85335    -0.689713
  1.07896     0.525982    0.289118   …   0.0567823   0.55163    -0.188831
 -0.571788    0.894983   -1.01287        0.231635    0.897395   -1.47794
  0.410177   -0.646728    0.0986629     -0.391472   -0.520274    0.816262
 -0.13357     0.943317   -0.434722      -0.821167    0.191804    0.169102
 -0.521011    1.86507    -1.1001         0.674711   -0.882319   -0.995623
  0.444361   -0.886811   -1.29127    …  -0.496684    0.0297876  -0.845236
  0.672564   -0.45112     0.374471       0.402569   -1.18894    -0.610256
 -1.09579    -0.935802    1.67602        0.553494   -0.928938    0.712325
  ⋮               

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [39]:
using NLopt

In [45]:
function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y) 
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    @show sigma
    @show negLog
    return negLog
end

negLogLik (generic function with 1 method)

In [53]:
function negLogLik(sigma::Vector{Float64})#, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

negLogLik (generic function with 2 methods)

In [56]:
#negLogLik([1.0,1.0], [2.0,2.0])
negLogLik([1.0,1.0])

sigma = [1.0, 1.0]
negLog = -136.2167523160515


-136.2167523160515

In [57]:
opt = Opt(:LN_COBYLA, 2)
opt.min_objective = negLogLik
opt.xtol_rel = 1e-5

1.0e-5

In [58]:
sigma = [1,1]
println("OPTBL: starting point $(sigma)")
(optf,optx,ret) = optimize(opt, sigma)
numevals = opt.numevals
println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $numevals iterations (returned $(ret))")

OPTBL: starting point [1, 1]
sigma = [1.0, 1.0]
negLog = -136.2167523160515
sigma = [2.0, 1.0]
negLog = -134.95427999075937
sigma = [1.0, 2.0]
negLog = -131.45602844837435
sigma = [0.7436747239790107, 0.03340941817501464]
negLog = -849.5243768369603
got -849.52438 at [0.74367, 0.03341] after 5 iterations (returned FORCED_STOP)


After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [ ]:
sigma = [1, 1]
n = 9

In [ ]:
Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))

In [ ]:
# A is X in formula
inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y

In [59]:
n = length(y)
Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
#println("OPT: parameter $(sigma) || objective eval $(negLog)")
@show sigma
@show negLog

sigma = [1, 1]
negLog = -136.2167523160515


-136.2167523160515

In [ ]:
using NLopt

function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end
    return sqrt(x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2]
end

opt = Opt(:LD_MMA, 2)
opt.lower_bounds = [-Inf, 0.]
opt.xtol_rel = 1e-4

opt.min_objective = myfunc
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,-1,1), 1e-8)

minf,minx,ret = optimize(opt, [1.234, 5.678])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

In [ ]:
opt.

In [ ]:
function test()
    x = 1+2
    y =  2*x
    z = [1,2]
    return (x,y,z)
end

In [ ]:
test()

In [ ]:
x

In [ ]:
y

### test fit!()

In [ ]:
#include("HighDimMixedModels.jl")
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

In [ ]:
#HighDimMM.fit!(HMM, verbose = true, REML = true)

In [ ]:
#HighDimMM.test!(HMM, verbose = true, REML = true)

In [ ]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
C = transpose(u[:,1:(n-r)]) 
K = C*P
Z = HMM.Z.Z
y = HMM.y

In [ ]:
function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    return negLog
end

In [ ]:
negLogLik([2.0,2.0],[2.0,2.0])

In [ ]:
using NLopt
opt = Opt(:LN_COBYLA, 2) # :GN_DIRECT_L  :LN_COBYLA :LN_BOBYQA
opt.min_objective = negLogLik
opt.xtol_rel = 1e-5

sigma = [2,2]
println("OPTBL: starting point $(sigma)")



In [ ]:
(optf,optx,ret) = optimize(opt, sigma)

In [ ]:
println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $numevals iterations (returned $(ret))") 

In [ ]:
ndims(opt)

In [ ]:
opt.algorithm

In [ ]:
alg = :LN_COBYLA
opt = Opt(alg, 2) # :GN_DIRECT_L  :LN_COBYLA :LN_BOBYQA
opt.min_objective = negLogLik
opt.xtol_rel = 1e-5

sigma = [2,2]
(optf,optx,ret) = optimize(opt, sigma)
println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $numevals iterations (returned $(ret))") 

In [ ]:
algorithm_name(opt::Opt)

In [ ]:
typeof(:LN_COBYLA)